In [1]:
#First please run this cell and fill in your google credentials (If worried I suppose you can use 
#a burner account but this opens a private client into your own gsutil 
#that will allow you to get access to the public directories 
#of my storage bucket in which I have stored very large model files) this is a great way to store large
#files pretty much for free if anyone is interested
!git clone https://github.com/aidan-collins/scene-bot
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers==2.8.0
!pip install tensorboardX
!pip install seqeval
!pip install gsutil
!gsutil config

Cloning into 'scene-bot'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 60 (delta 15), reused 51 (delta 13), pack-reused 0
Unpacking objects: 100% (60/60), done.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 23kB/s 
     |████████████████████████████████| 5.9MB 35.0MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 573kB 9.1MB/s 
     |████████████████████████████████| 3.7MB 15.6MB/s 
     |████████████████████████████████| 1.1MB 63.1MB/s 
     |████████████████████████████████| 890kB 62.4MB/s 
     |████████████████████████████████| 133k

In [1]:
!gsutil -m cp -r gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim /content
!gsutil -m cp -r gs://bucket-o-words/gedi-runs/run-1/topic_scripts /content
!gsutil -m cp -r gs://bucket-o-words/gedi-runs/run-3/topic_scripts_genres_3_small /content
!gsutil -m cp -r gs://bucket-o-words/final-movie-language-model/base_writer-dialogpt2-small-five /content
!mv /content/topic_scripts /content/scene-bot/GeDi
!mv /content/topic_scripts_genres_3_small /content/scene-bot/GeDi
!mv /content/final-writing-model-sim /content/scene-bot/GeDi
!mv /content/base_writer-dialogpt2-small-five /content/scene-bot/GeDi


Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/config.json...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/merges.txt...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/optimizer.pt...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/pytorch_model.bin...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/scheduler.pt...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/special_tokens_map.json...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/special_tokens_map.json...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-model-sim/writer-model/checkpoint-108500/trainer_state.json...
Copying gs://bucket-o-words/gpt2-medium-runs/final-writing-m

In [12]:
#Experiment 1 finetuned GeDi (from Dialogpt-small base model) on seq len ~200 batching of 8 -> superior performance to larger model, seq_len, batch
%cd
%cd /content/scene-bot/GeDi
import torch
import numpy as np 

from modeling_gpt2 import GPT2LMHeadModel

from transformers import ( GPT2Config, GPT2Tokenizer )
mode = "topic"
code_desired = "true"
code_undesired = "false"
model_type = 'gpt2'
gen_type = "gedi"
gen_model_name_or_path = "microsoft/DialoGPT-large"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),}
config_class, model_class, tokenizer_class = MODEL_CLASSES["gpt2"]
tokenizer = tokenizer_class.from_pretrained('microsoft/DialoGPT-large', do_lower_case=False) #we'll grab a copy of the public tokenizer as it was ot altered during training

model = model_class.from_pretrained(gen_model_name_or_path)
model = model.to(device)
model = model.float()

gedi_model_name_or_path = '/content/scene-bot/GeDi/topic_scripts'
gedi_model = model_class.from_pretrained(gedi_model_name_or_path)
gedi_model.to(device)

#setting arguments for generation
#max generation length
gen_length = 200
#omega from paper, higher disc_weight means more aggressive topic steering
disc_weight = 30
#1 - rho from paper, should be between 0 and 1 higher filter_p means more aggressive topic steering
filter_p = 0.8
#tau from paper, preserves tokens that are classified as correct topic
target_p = 0.8
#hyperparameter that determines class prior, set to uniform by default
class_bias = 0

if gen_length>1024:
  length = 1024
else:
  length = gen_length


print('Please input a movie genre to select from (try sci-fi, horror, comedy and romance for example).')
secondary_code = 'sci-fi'
secondary_code = input('Movie genre: ')
bpe_tokens = tokenizer.encode(secondary_code)



multi_code = tokenizer.encode(secondary_code)
attr_class = 1

text = ""
num_turns = 5 #generation lengths are unexpected so no garuntees youll get to the end of your conversation
for i in range(num_turns):
  prompt = input('Your turn: ')
  prompt = text + " " + prompt + " "  + tokenizer.eos_token
  text_ids = tokenizer.encode(prompt)
  encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0).to(device)

  generated_sequence = model.generate(input_ids=encoded_prompts,
                                          pad_lens=None,
                                            max_length= length,
                                            top_k=None,
                                            top_p=None,
                                            repetition_penalty= 1.2,
                                            rep_penalty_scale= 10,
                                            eos_token_ids = tokenizer.eos_token_id,
                                            pad_token_id = 0,
                                            do_sample= False,
                                            penalize_cond= True,
                                            gedi_model= gedi_model,
                                            tokenizer= tokenizer,
                                            disc_weight= disc_weight,
                                            filter_p = filter_p,
                                            target_p = target_p,
                                            class_bias = class_bias,
                                            attr_class = attr_class,
                                            code_0 = code_undesired,
                                            code_1 = code_desired,
                                            multi_code=multi_code
                                            )

  text =  tokenizer.decode(generated_sequence.tolist()[0], clean_up_tokenization_spaces=True)
  print('\n')
  print(text)


/root
/content/scene-bot/GeDi



no logit scale initialized for gpt2
Please input a movie genre to select from (try sci-fi, horror, comedy and romance for example).
Movie genre: horror
Your turn: hello script writer what are you doing?
GeDi estimates the probability that it sample is desired class is: 0.5077186822891235


 hello script writer what are you doing?<|endoftext|>Writing scripts.<|endoftext|>
Your turn: oh naturally... thats your job
GeDi estimates the probability that it sample is desired class is: 0.505055844783783


 hello script writer what are you doing?<|endoftext|>Writing scripts.<|endoftext|> oh naturally... thats your job<|endoftext|>I write the scripts myself actually. I'm a script editor and wrote the first one in my class last semester. It was pretty fun writing it all out.<|endoftext|>


KeyboardInterrupt: ignored

In [ ]:
#Experiment 2 finetuned GeDi (from Dialogpt-medium base model) on seq len ~700 batching of 2 -> inferior performance to smaller model, seq_len, batch
%cd
%cd /content/scene-bot/GeDi
import torch
import numpy as np 

from modeling_gpt2 import GPT2LMHeadModel

from transformers import ( GPT2Config, GPT2Tokenizer )
mode = "topic"
code_desired = "true"
code_undesired = "false"
model_type = 'gpt2'
gen_type = "gedi"
gen_model_name_or_path = "microsoft/DialoGPT-large"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),}
config_class, model_class, tokenizer_class = MODEL_CLASSES["gpt2"]
tokenizer = tokenizer_class.from_pretrained('microsoft/DialoGPT-large', do_lower_case=False) #we'll grab a copy of the public tokenizer as it was ot altered during training

model = model_class.from_pretrained(gen_model_name_or_path)
model = model.to(device)
model = model.float()

gedi_model_name_or_path = 'topic_scripts_genres_3_small'
gedi_model = model_class.from_pretrained(gedi_model_name_or_path)
gedi_model.to(device)

#setting arguments for generation
#max generation length
gen_length = 200
#omega from paper, higher disc_weight means more aggressive topic steering
disc_weight = 30
#1 - rho from paper, should be between 0 and 1 higher filter_p means more aggressive topic steering
filter_p = 0.8
#tau from paper, preserves tokens that are classified as correct topic
target_p = 0.8
#hyperparameter that determines class prior, set to uniform by default
class_bias = 0

if gen_length>1024:
  length = 1024
else:
  length = gen_length


print('Please input a movie genre to select from (try sci-fi, horror, comedy and romance for example).')
secondary_code = 'sci-fi'
secondary_code = input('Movie genre: ')
bpe_tokens = tokenizer.encode(secondary_code)



multi_code = tokenizer.encode(secondary_code)
attr_class = 1

text = ""
num_turns = 5 #generation lengths are unexpected so no garuntees youll get to the end of your conversation
for i in range(num_turns):
  prompt = input('Your turn: ')
  prompt = text + " " + prompt + " "  + tokenizer.eos_token
  text_ids = tokenizer.encode(prompt)
  encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0).to(device)

  generated_sequence = model.generate(input_ids=encoded_prompts,
                                          pad_lens=None,
                                            max_length= length,
                                            top_k=None,
                                            top_p=None,
                                            repetition_penalty= 1.2,
                                            rep_penalty_scale= 10,
                                            eos_token_ids = tokenizer.eos_token_id,
                                            pad_token_id = 0,
                                            do_sample= False,
                                            penalize_cond= True,
                                            gedi_model= gedi_model,
                                            tokenizer= tokenizer,
                                            disc_weight= disc_weight,
                                            filter_p = filter_p,
                                            target_p = target_p,
                                            class_bias = class_bias,
                                            attr_class = attr_class,
                                            code_0 = code_undesired,
                                            code_1 = code_desired,
                                            multi_code=multi_code
                                            )

  text =  tokenizer.decode(generated_sequence.tolist()[0], clean_up_tokenization_spaces=True)
  print('\n')
  print(text)


In [10]:
%cd
%cd /content/scene-bot/GeDi
import torch
import numpy as np 

from modeling_gpt2 import GPT2LMHeadModel

from transformers import ( GPT2Config, GPT2Tokenizer )
mode = "topic"
code_desired = "true"
code_undesired = "false"
model_type = 'gpt2'
gen_type = "gedi"
gen_model_name_or_path = "/content/scene-bot/GeDi/final-writing-model-sim/writer-model"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),}
config_class, model_class, tokenizer_class = MODEL_CLASSES["gpt2"]
tokenizer = tokenizer_class.from_pretrained('microsoft/DialoGPT-small', do_lower_case=False) #we'll grab a copy of the public tokenizer as it was ot altered during training

model = model_class.from_pretrained(gen_model_name_or_path)
model = model.to(device)
model = model.float()

gedi_model_name_or_path = '/content/scene-bot/GeDi/topic_scripts'
gedi_model = model_class.from_pretrained(gedi_model_name_or_path)
gedi_model.to(device)

#setting arguments for generation
#max generation length
gen_length = 200
#omega from paper, higher disc_weight means more aggressive topic steering
disc_weight = 30
#1 - rho from paper, should be between 0 and 1 higher filter_p means more aggressive topic steering
filter_p = 0.8
#tau from paper, preserves tokens that are classified as correct topic
target_p = 0.8
#hyperparameter that determines class prior, set to uniform by default
class_bias = 0

if gen_length>1024:
  length = 1024
else:
  length = gen_length


print('Please input a movie genre to select from (try sci-fi, horror, comedy and romance for example).')
secondary_code = 'sci-fi'
secondary_code = input('Movie genre: ')
bpe_tokens = tokenizer.encode(secondary_code)



multi_code = tokenizer.encode(secondary_code)
attr_class = 1

text = ""
num_turns = 5 #generation lengths are unexpected so no garuntees youll get to the end of your conversation
for i in range(num_turns):
  prompt = input('Your turn: ')
  prompt = text + " " + prompt + " "  + tokenizer.eos_token
  text_ids = tokenizer.encode(prompt)
  encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0).to(device)

  generated_sequence = model.generate(input_ids=encoded_prompts,
                                          pad_lens=None,
                                            max_length= length,
                                            top_k=None,
                                            top_p=None,
                                            repetition_penalty= 1.2,
                                            rep_penalty_scale= 10,
                                            eos_token_ids = tokenizer.eos_token_id,
                                            pad_token_id = 0,
                                            do_sample= False,
                                            penalize_cond= True,
                                            gedi_model= gedi_model,
                                            tokenizer= tokenizer,
                                            disc_weight= disc_weight,
                                            filter_p = filter_p,
                                            target_p = target_p,
                                            class_bias = class_bias,
                                            attr_class = attr_class,
                                            code_0 = code_undesired,
                                            code_1 = code_desired,
                                            multi_code=multi_code
                                            )

  text =  tokenizer.decode(generated_sequence.tolist()[0], clean_up_tokenization_spaces=True)
  print('\n')
  print(text)


/root
/content/scene-bot/GeDi
no logit scale initialized for gpt2
Please input a movie genre to select from (try sci-fi, horror, comedy and romance for example).
Movie genre: sci-fi
Your turn: hello folks
GeDi estimates the probability that it sample is desired class is: 0.475144624710083


 hello folks<|endoftext|>Hiya Jimmy. <|endoftext|>
Your turn: my names bert
GeDi estimates the probability that it sample is desired class is: 0.48719778656959534


 hello folks<|endoftext|>Hiya Jimmy.<|endoftext|> my names bert<|endoftext|> Hiya Teddy. <|endoftext|>
Your turn: ok asshole
GeDi estimates the probability that it sample is desired class is: 0.4918922185897827


 hello folks<|endoftext|>Hiya Jimmy.<|endoftext|> my names bert<|endoftext|> Hiya Teddy.<|endoftext|> ok asshole<|endoftext|> Teddy meet my brother Gabe. <|endoftext|>
Your turn: i dont want to meet gabe
GeDi estimates the probability that it sample is desired class is: 0.49781450629234314


 hello folks<|endoftext|>Hiya Jimmy.<

KeyboardInterrupt: ignored